In [77]:
import grpc
import json
from google.protobuf import text_format, json_format
from google.protobuf.json_format import MessageToJson
from tensorflow_serving.apis import model_service_pb2_grpc, model_management_pb2
from tensorflow_serving.config import model_server_config_pb2

def get_grpc_connection(host, port=443, crt_path="server.crt"):
    """Establish secure grpc channel"""
    with open(crt_path, 'rb') as f:
        trusted_certs = f.read()
        
    credentials = grpc.ssl_channel_credentials(root_certificates=trusted_certs)
    channel = grpc.secure_channel('{}:{}'.format(host, port), credentials)
    return channel

def add_model_config(conf_filepath, host, port, crt_path, name, base_path, model_platform):

    # Merge new server config into existing
    with open(conf_filepath, 'r') as openfile:
        config = json.load(openfile)
               
    config["modelConfigList"]["config"].append(
        {
            "name": name,
            "base_path": base_path,
            "model_platform": model_platform
        })
    
    result = {}
    for c in config["modelConfigList"]["config"]:
        result.setdefault(c["name"], {}).update(c)
    config["modelConfigList"]["config"] = list(result.values())

    # Load proto buffers
    channel = get_grpc_connection(host, port, crt_path)
    
    stub = model_service_pb2_grpc.ModelServiceStub(channel)
    request = model_management_pb2.ReloadConfigRequest()
    model_server_config = model_server_config_pb2.ModelServerConfig()
    config_list = model_server_config_pb2.ModelConfigList()
        
    # Parse json config to proto message
    model_server_config = json_format.Parse(text=json.dumps(config), message=model_server_config)

    request.config.CopyFrom(model_server_config)
    with open(conf_filepath, "w") as outfile:
        outfile.write(json.dumps(json.loads(MessageToJson(request.config)), indent=4))

    #response = stub.HandleReloadConfigRequest(request, 10)
    #if response.status.error_code == 0:
    #    
    #    print(MessageToJson(request.config))
    #    print("Updated TF Serving conf file")
    #else:
    #    print("Failed to update model_config_list!")
    #    print(response.status.error_code)
    #    print(response.status.error_message)

In [83]:
crt_path = "./cert/server.crt"
host="model-server-grpc.192.168.1.203.nip.io"
port="443"

add_model_config("./model_config.json", host, port, crt_path, "quad2", "/models/quad2/", "tensorflow")


{'modelConfigList': {'config': [{'name': 'mlmodel', 'basePath': '/models/mlmodel/', 'modelPlatform': 'tensorflow'}, {'name': 'quad', 'basePath': '/models/quad/', 'modelPlatform': 'tensorflow'}, {'name': 'quad2', 'basePath': '/models/quad/', 'modelPlatform': 'tensorflow', 'base_path': '/models/quad2/', 'model_platform': 'tensorflow'}]}}


In [32]:
jsonString = {
  "modelConfigList": {
    "config": [
      {
        "name": "mlmodel",
        "basePath": "/models/mlmodel/",
        "modelPlatform": "tensorflow"
      },
      {
        "name": "quad",
        "basePath": "/models/quad/",
        "modelPlatform": "tensorflow"
      },
      {
        "name": "quad2",
        "basePath": "/models/quad/",
        "modelPlatform": "tensorflow"
      }
    ]
  }
}

In [102]:
import json
json.dumps({"list": x.values.tolist()})


'{"list": [[1, 2, 3], [2, 3, 4]]}'

In [106]:
import numpy as np
y = np.array(x.values.tolist())

In [111]:
y.shape[1]

3

In [119]:
z = pd.DataFrame(x.values, columns=[f'y_{y}' for y in range(y.shape[1])])

In [156]:
z[["y_0"]].values

array([[1],
       [2]])

In [146]:
z["y_0"].values.shape

(2,)